# 基于OLS的疫情和GDP相关性分析
>利用2020年和2021年各季度GDP增长和疫情相关数据进行相关性分析


## 模块导入

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns #统计绘图
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm
from scipy import stats #统计
from statsmodels.formula.api import ols
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## 文件读取

In [3]:
gdpData = pd.read_csv('/home/mw/input/gmjj6646/gmjj/gmjj/GDP.csv')
covidData = pd.read_csv('/home/mw/input/covid199004/owid-covid-data.csv')
chinaData = covidData.loc[covidData['location']=='China']
gdpData.info()
covidData.info()
chinaData.info()
print(chinaData)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 9 columns):
Quater                   64 non-null object
GDP_Absolute             64 non-null float64
GDP_YOY                  64 non-null object
Primary_Indusry_Abs      64 non-null float64
Primary_Indusry_YOY      64 non-null object
Secondary_Indusry_Abs    64 non-null float64
Secondary_Indusry_YOY    64 non-null object
Tertiary_Indusry_Abs     64 non-null float64
Tertiary_Indusry_YOY     64 non-null object
dtypes: float64(4), object(5)
memory usage: 4.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167708 entries, 0 to 167707
Data columns (total 67 columns):
iso_code                                      167708 non-null object
continent                                     157674 non-null object
location                                      167708 non-null object
date                                          167708 non-null object
total_cases                                   164661 non-nu

## 数据处理

>数据列表
* 2020-2021年季度生产总值绝对值
* 2020-2021年季度第一产业绝对值
* 2020-2021年季度第二产业绝对值
* 2020-2021年季度第三产业绝对值
* 2020-2021年季度新增确诊的 COVID-19 病例人数
* 2020-2021年季度新增归因于 COVID-19死亡人数
* 2020-2021年季度新增的 COVID-19 疫苗接种剂量
* 2020-2021年季度的 COVID-19 疫苗接种剂量
* 2020-2021年季度平均政府响应严格度指数


### 2020-2021年各季度生产总值绝对值数据

In [4]:
gdp = gdpData['GDP_Absolute'][:8]
gdp = gdp.tolist()
gdp.reverse()
GDP_Season = [i for i in gdp]

for i in range(len(GDP_Season)):
    if (i%4) != 0 :
        GDP_Season[i] -=  gdp[i-1]
    
GDP_Season

[205727.0,
 248985.09999999998,
 264976.30000000005,
 296297.79999999993,
 249310.0,
 282857.0,
 290964.0,
 320539.0]

### 2020-2021年各季度第一产业绝对值

In [5]:
industry1 = gdpData['Primary_Indusry_Abs'][:8]
industry1 = industry1.tolist()
industry1.reverse()
Primary_Indusry_Season = [i for i in industry1]

for i in range(len(industry1)):
    if (i%4) != 0 :
        Primary_Indusry_Season[i] -=  industry1[i-1]

Primary_Indusry_Season

[10185.1,
 15866.800000000001,
 22072.0,
 29630.200000000004,
 11332.0,
 17069.0,
 23029.0,
 31656.0]

### 2020-2021年各季度第二产业绝对值

In [6]:
industry2 = gdpData['Secondary_Indusry_Abs'][:8]
industry2 = industry2.tolist()
industry2.reverse()
Secondary_Indusry_Season = [i for i in industry2]

for i in range(len(industry2)):
    if (i%4) != 0 :
        Secondary_Indusry_Season[i] -=  industry2[i-1]

Secondary_Indusry_Season

[72533.4,
 97699.4,
 100082.60000000003,
 113939.89999999997,
 92623.0,
 114531.0,
 113786.0,
 129964.0]

### 2020-2021年各季度第三产业绝对值

In [7]:
industry3 = gdpData['Tertiary_Indusry_Abs'][:8]
industry3 = industry3.tolist()
industry3.reverse()
Tertiary_Indusry_Season = [i for i in industry3]

for i in range(len(industry3)):
    if (i%4) != 0 :
        Tertiary_Indusry_Season[i] -=  industry3[i-1]

Tertiary_Indusry_Season

[123008.5,
 135418.9,
 142821.69999999998,
 152727.70000000007,
 145355.0,
 151256.0,
 154150.0,
 158919.0]

### 2020-2021年各季度疫情相关数据

In [8]:
chinaData = covidData.loc[covidData['location']=='China'].fillna(0)

In [9]:
arr = [i for i in range(0, 721, 90)]

New_Cases_Season = []
New_Deaths_Season = []
New_Vaccinations_Season = []
Total_Vaccinations = []
Stringency_Index = []
for i in range(len(arr)-1):
    New_Cases_Season.append(sum(chinaData['new_cases'][:arr[i+1]])-sum(chinaData['new_cases'][:arr[i]]))
    New_Deaths_Season.append(sum(chinaData['new_deaths'][:arr[i+1]])-sum(chinaData['new_deaths'][:arr[i]]))
    New_Vaccinations_Season.append(sum(chinaData['new_vaccinations'][:arr[i+1]])-sum(chinaData['new_vaccinations'][:arr[i]]))
    Total_Vaccinations.append(sum(chinaData['new_vaccinations'][:arr[i+1]])/arr[i+1])
    Stringency_Index.append((sum(chinaData['stringency_index'][:arr[i+1]])-sum(chinaData['stringency_index'][:arr[i]]))/90)

print(New_Cases_Season)
print(New_Deaths_Season)
print(New_Vaccinations_Season)
print(Total_Vaccinations)
print(Stringency_Index)

[82200.0, 934.0, 1992.0, 2446.0, 2350.0, 1679.0, 4315.0, 7511.0]
[4615.0, 2.0, 0.0, 1.0, 1.0, 0.0, 3.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 103076000.0, 1231073000.0, 809472000.0, 681557000.0]
[0.0, 0.0, 0.0, 0.0, 229057.77777777778, 2470646.296296296, 3402573.015873016, 3923858.3333333335]
[72.42733333333331, 74.73000000000002, 68.09633333333332, 74.47466666666668, 66.99033333333334, 72.42711111111106, 72.99166666666666, 72.03633333333337]


### 数据合并

In [10]:
Stringency_Index.reverse()
data = pd.DataFrame({'GDP_Season':GDP_Season,'Primary_Indusry_Season':Primary_Indusry_Season,
                'Secondary_Indusry_Season':Secondary_Indusry_Season,'Tertiary_Indusry_Season':Tertiary_Indusry_Season,
                'New_Cases_Season':New_Cases_Season,'New_Deaths_Season':New_Deaths_Season,
                'New_Vaccinations_Season':New_Vaccinations_Season,'Total_Vaccinations':Total_Vaccinations,
                'Stringency_Index':Stringency_Index})
data

,GDP_Season,Primary_Indusry_Season,Secondary_Indusry_Season,Tertiary_Indusry_Season,New_Cases_Season,New_Deaths_Season,New_Vaccinations_Season,Total_Vaccinations,Stringency_Index
0,205727.0,10185.1,72533.4,123008.5,82200.0,4615.0,0.000000e+00,0.000000e+00,72.036333
1,248985.1,15866.8,97699.4,135418.9,934.0,2.0,0.000000e+00,0.000000e+00,72.991667
2,264976.3,22072.0,100082.6,142821.7,1992.0,0.0,0.000000e+00,0.000000e+00,72.427111
3,296297.8,29630.2,113939.9,152727.7,2446.0,1.0,0.000000e+00,0.000000e+00,66.990333
4,249310.0,11332.0,92623.0,145355.0,2350.0,1.0,1.030760e+08,2.290578e+05,74.474667
5,282857.0,17069.0,114531.0,151256.0,1679.0,0.0,1.231073e+09,2.470646e+06,68.096333
6,290964.0,23029.0,113786.0,154150.0,4315.0,3.0,8.094720e+08,3.402573e+06,74.730000
7,320539.0,31656.0,129964.0,158919.0,7511.0,0.0,6.815570e+08,3.923858e+06,72.427333


## 回归分析

> 利用最小二乘法OLS分析多因素对各季度生产总值绝对值的影响
### 因素列表
* 2020-2021年季度新增确诊的 COVID-19 病例人数
* 2020-2021年季度新增归因于 COVID-19死亡人数
* 2020-2021年季度新增的 COVID-19 疫苗接种剂量
* 2020-2021年季度的 COVID-19 疫苗接种剂量
* 2020-2021年季度平均政府响应严格度指数



In [11]:
feature_data = data.drop(['GDP_Season'],axis=1)
target_data = data['GDP_Season']
df_train = pd.concat([feature_data,target_data],axis=1)

lr_model = ols("GDP_Season~New_Cases_Season+New_Deaths_Season+New_Vaccinations_Season+Total_Vaccinations+Stringency_Index",data=df_train).fit()
print(lr_model.summary())

                            OLS Regression Results                            
Dep. Variable:             GDP_Season   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.966
Method:                 Least Squares   F-statistic:                     40.72
Date:                Tue, 10 May 2022   Prob (F-statistic):             0.0241
Time:                        14:28:43   Log-Likelihood:                -76.129
No. Observations:                   8   AIC:                             164.3
Df Residuals:                       2   BIC:                             164.7
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                6

## 计算数据间的相关系数


In [12]:
corrmatrix = data.corr()
corrmatrix

,GDP_Season,Primary_Indusry_Season,Secondary_Indusry_Season,Tertiary_Indusry_Season,New_Cases_Season,New_Deaths_Season,New_Vaccinations_Season,Total_Vaccinations,Stringency_Index
GDP_Season,1.000000,0.888825,0.989159,0.960859,-0.686261,-0.728900,0.539298,0.695358,-0.257554
Primary_Indusry_Season,0.888825,1.000000,0.845584,0.762103,-0.461745,-0.506168,0.208069,0.481564,-0.311756
Secondary_Indusry_Season,0.989159,0.845584,1.000000,0.940846,-0.693253,-0.734175,0.609257,0.734861,-0.260723
Tertiary_Indusry_Season,0.960859,0.762103,0.940846,1.000000,-0.738237,-0.776669,0.588309,0.690067,-0.182319
New_Cases_Season,-0.686261,-0.461745,-0.693253,-0.738237,1.000000,0.997286,-0.268575,-0.239890,0.055228
New_Deaths_Season,-0.728900,-0.506168,-0.734175,-0.776669,0.997286,1.000000,-0.294118,-0.295421,0.038495
New_Vaccinations_Season,0.539298,0.208069,0.609257,0.588309,-0.268575,-0.294118,1.000000,0.850744,-0.147731
Total_Vaccinations,0.695358,0.481564,0.734861,0.690067,-0.239890,-0.295421,0.850744,1.000000,0.124368
Stringency_Index,-0.257554,-0.311756,-0.260723,-0.182319,0.055228,0.038495,-0.147731,0.124368,1.000000


## 热力图绘制

In [13]:
pylab.rcParams['figure.figsize'] = (15, 15)

# 绘制热力图，热力图横纵坐标分别是data的index/column,vmax/vmin设置热力图颜色标识上下限，center显示颜色标识中心位置，cmap颜色标识颜色设置
s1=sns.heatmap(corrmatrix,square=True,vmax=1,vmin=-1,center=0.0,cmap='coolwarm')
s1 = s1.get_figure()
s1.savefig('HeatMap.jpg',dpi=300,bbox_inches='tight')

<Figure size 1080x1080 with 2 Axes>